In [ ]:
import pandas as pd
import numpy as np
import torch
import playwright
import jmespath
from playwright.sync_api import sync_playwright
from twikit import Client
import json
import transformers
import sentencepiece
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

The command to use in order to have access to a tweet with its id is :

``twitter.com/anyuser/status/541278904204668929
``
With 541278904204668929 being an example of a tweet's id.

In [ ]:
url = 'https://raw.githubusercontent.com/patriChiril/An-Annotated-Corpus-for-Sexism-Detection-in-French-Tweets/refs/heads/master/corpus_SexistContent.csv'
df_base = pd.read_csv(url, sep='\t')

In [ ]:
df_base.columns = ['id','label']

In [ ]:
df_base.to_csv('data_id.csv')

In [ ]:
url = 'https://docs.google.com/spreadsheets/d/1eoC3NooDOwFoOvfweVLEZ-z-jCPZghUCVMUPTBkUDIk/export?format=csv&gid=1834088666'
df_whole = pd.read_csv(url)

In [ ]:
df_whole.head()

### Model's import

In [ ]:
from transformers import CamembertModel, CamembertTokenizer

In [ ]:
test_str = 'tarte au prout et au caca'

In [ ]:
# Load model directly
from transformers import CamembertModel, CamembertTokenizer

# You can replace "camembert-base" with any other model from the table, e.g. "camembert/camembert-large".
tokenizer = CamembertTokenizer.from_pretrained("camembert/camembert-base-wikipedia-4gb")
camembert = CamembertModel.from_pretrained("camembert/camembert-base-wikipedia-4gb")

camembert.eval()  # disable dropout (or leave in train mode to finetune)


In [ ]:
tokenized_sentence = tokenizer.tokenize('tarte au caca')
encoded_sentence = tokenizer.encode(tokenized_sentence)

In [ ]:
encoded_sentence = torch.tensor(encoded_sentence).unsqueeze(0)
embeddings, _ = camembert(encoded_sentence)

In [ ]:
embeddings.detach()

In [ ]:
encoded_sentence = tokenizer.encode(tokenized_sentence)
# [5, 221, 10, 10600, 14, 8952, 10540, 75, 1114, 6]
# NB: Can be done in one step : tokenize.encode("J'aime le camembert !")

# Feed tokens to Camembert as a torch tensor (batch dim 1)
encoded_sentence = torch.tensor(encoded_sentence).unsqueeze(0)
embeddings, _ = camembert(encoded_sentence)

In [ ]:
embeddings

In [ ]:
import torch
# Tokenize in sub-words with SentencePiece
tokenized_sentence = tokenizer.tokenize("J'aime le camembert !")
# ['▁J', "'", 'aime', '▁le', '▁ca', 'member', 't', '▁!'] 

# 1-hot encode and add special starting and end tokens 
encoded_sentence = tokenizer.encode(tokenized_sentence)
# [5, 221, 10, 10600, 14, 8952, 10540, 75, 1114, 6]
# NB: Can be done in one step : tokenize.encode("J'aime le camembert !")

# Feed tokens to Camembert as a torch tensor (batch dim 1)
encoded_sentence = torch.tensor(encoded_sentence).unsqueeze(0)
embeddings, _ = camembert(encoded_sentence)
# embeddings.detach()
# embeddings.size torch.Size([1, 10, 768])
#tensor([[[-0.0928,  0.0506, -0.0094,  ..., -0.2388,  0.1177, -0.1302],
#         [ 0.0662,  0.1030, -0.2355,  ..., -0.4224, -0.0574, -0.2802],
#         [-0.0729,  0.0547,  0.0192,  ..., -0.1743,  0.0998, -0.2677],
#         ...,


In [ ]:
embeddings

In [ ]:
from transformers import CamembertModel, CamembertTokenizer

# You can replace "camembert-base" with any other model from the table, e.g. "camembert/camembert-large".
tokenizer = CamembertTokenizer.from_pretrained("camembert/camembert-base-wikipedia-4gb")
camembert = CamembertModel.from_pretrained("camembert/camembert-base-wikipedia-4gb")

camembert.eval()  # disable dropout (or leave in train mode to finetune)

In [ ]:
import torch
# Tokenize in sub-words with SentencePiece
tokenized_sentence = tokenizer.tokenize("J'aime le camembert !")
# ['▁J', "'", 'aime', '▁le', '▁ca', 'member', 't', '▁!'] 

# 1-hot encode and add special starting and end tokens 
encoded_sentence = tokenizer.encode(tokenized_sentence)
# [5, 221, 10, 10600, 14, 8952, 10540, 75, 1114, 6]
# NB: Can be done in one step : tokenize.encode("J'aime le camembert !")

# Feed tokens to Camembert as a torch tensor (batch dim 1)
encoded_sentence = torch.tensor(encoded_sentence).unsqueeze(0)
embeddings, _ = camembert(encoded_sentence)
# embeddings.detach()
# embeddings.size torch.Size([1, 10, 768])
#tensor([[[-0.0928,  0.0506, -0.0094,  ..., -0.2388,  0.1177, -0.1302],
#         [ 0.0662,  0.1030, -0.2355,  ..., -0.4224, -0.0574, -0.2802],
#         [-0.0729,  0.0547,  0.0192,  ..., -0.1743,  0.0998, -0.2677],
#         ...,


In [ ]:
embeddings

In [ ]:
encoded_sentence

In [ ]:
import torch
from transformers import CamembertTokenizer, CamembertModel

# Load CamemBERT model and tokenizer
tokenizer = CamembertTokenizer.from_pretrained("camembert-base")
model = CamembertModel.from_pretrained("camembert-base")

# Input sentence
sentence = "J'aime le camembert !"

# Tokenize and convert to PyTorch tensors
inputs = tokenizer(sentence, return_tensors="pt", return_attention_mask=True)

# Forward pass (no gradients needed)
with torch.no_grad():
    outputs = model(**inputs)

# Get token embeddings from last hidden state: [1, seq_len, 768]
last_hidden_state = outputs.last_hidden_state

# Decode tokens to match embeddings with actual text tokens
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

# Print token-wise embeddings
for idx, token in enumerate(tokens):
    embedding = last_hidden_state[0, idx]  # shape: [768]
    print(f"Token: {token}\nEmbedding: {embedding.tolist()[:5]}...")  # first 5 dims for brevity


Exemple de bite en bois de merde : 

In [ ]:
import pandas as pd
import torch
from transformers import CamembertTokenizerFast, CamembertModel
tokenizer = CamembertTokenizerFast.from_pretrained("camembert-base")


# Set device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load model and tokenizer
tokenizer = CamembertTokenizerFast.from_pretrained("camembert-base")
model = CamembertModel.from_pretrained("camembert-base")
model.to(device)
model.eval()

# Sample DataFrame
df = pd.DataFrame({
    'text': [
        "J'aime le camembert !",
        "Les chats dorment sur le canapé.",
        "Il pleut souvent à Paris."
    ]
})

# Tokenize and batch the text
inputs = tokenizer(
    df['text'].tolist(),
    padding=True,
    truncation=True,
    return_tensors='pt',
    return_attention_mask=True,
    return_offsets_mapping=True,
    add_special_tokens=True
)

# Move inputs to GPU
input_ids = inputs['input_ids'].to(device)
attention_mask = inputs['attention_mask'].to(device)

# Run model on GPU
with torch.no_grad():
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)

# Get token embeddings (last hidden state)
token_embeddings = outputs.last_hidden_state  # shape: [batch_size, seq_len, 768]

# Optionally move embeddings back to CPU
token_embeddings_cpu = token_embeddings.cpu()

# Convert input IDs to tokens
batch_tokens = [
    tokenizer.convert_ids_to_tokens(seq_ids) for seq_ids in input_ids.cpu()
]

# Display tokens and their embeddings (first 3 dimensions shown for brevity)
for i, tokens in enumerate(batch_tokens):
    print(f"\nSentence {i + 1}:")
    for j, token in enumerate(tokens):
        vector = token_embeddings_cpu[i, j]
        print(f"  Token: {token.ljust(12)} Embedding: {vector[:3].tolist()}...")


Exemple de retrain a la con de merde : 

In [ ]:
import torch
import torch.nn as nn
from transformers import CamembertModel

class CamembertSentenceClassifier(nn.Module):
    def __init__(self, hidden_dim=768, mid_dim=256, dropout=0.1):
        super().__init__()

        # Load pre-trained CamemBERT
        self.bert = CamembertModel.from_pretrained("camembert-base")
        
        # Freeze all BERT parameters
        for param in self.bert.parameters():
            param.requires_grad = False

        # Classification head (only this is trainable)
        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim, mid_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(mid_dim, 1),      # Binary classification
            nn.Sigmoid()                # Output in [0, 1]
        )

    def forward(self, input_ids, attention_mask):
        with torch.no_grad():
            outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        
        # Use mean pooling over token embeddings to get sentence representation
        token_embeddings = outputs.last_hidden_state       # [batch_size, seq_len, 768]
        pooled = token_embeddings.mean(dim=1)              # [batch_size, 768]

        # Classifier outputs probability
        return self.classifier(pooled)                     # [batch_size, 1]


In [ ]:
# Instantiate model
model = CamembertSentenceClassifier()
model.to(device)

# Optimizer (only train classifier parameters)
optimizer = torch.optim.Adam(model.classifier.parameters(), lr=1e-4)

# Loss function for binary classification
loss_fn = nn.BCELoss()


In [ ]:
for batch in dataloader:
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device).float()  # or .long() for multi-class

    outputs = model(input_ids, attention_mask)

    # Use appropriate loss
    if model.is_binary:
        loss_fn = nn.BCELoss()
    else:
        loss_fn = nn.CrossEntropyLoss()

    loss = loss_fn(outputs.squeeze(), labels)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()


In [ ]:
import torch
import pandas as pd
from transformers import CamembertTokenizerFast, CamembertModel

# Load pre-trained model and tokenizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = CamembertTokenizerFast.from_pretrained("camembert-base")
model = CamembertModel.from_pretrained("camembert-base").to(device)
model.eval()

# Sample DataFrame
df = pd.DataFrame({
    'text': [
        "J'aime le camembert !",
        "Les chats dorment sur le canapé.",
        "Il pleut souvent à Paris."
    ]
})

# Tokenize and encode all sentences at once
inputs = tokenizer(
    df['text'].tolist(),
    padding=True,
    truncation=True,
    return_tensors='pt',
    return_attention_mask=True
)

# Move tensors to GPU
input_ids = inputs['input_ids'].to(device)
attention_mask = inputs['attention_mask'].to(device)

# Generate embeddings (with no gradients)
with torch.no_grad():
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)

# Extract sentence embeddings by averaging token embeddings (mean pooling)
token_embeddings = outputs.last_hidden_state  # [batch_size, seq_len, hidden]
sentence_embeddings = token_embeddings.mean(dim=1)  # [batch_size, hidden_dim]

# Convert to CPU for further use
sentence_embeddings_cpu = sentence_embeddings.cpu().numpy()


In [ ]:
# Add labels (for demonstration, you'll need to replace this with your own labels)
df['label'] = [1, 0, 1]  # Example labels, modify accordingly

# Features (sentence embeddings) and Labels
X = sentence_embeddings_cpu  # Embeddings
y = df['label'].values       # Labels (0 or 1)

# Convert to tensors for PyTorch
X_tensor = torch.tensor(X).float().to(device)
y_tensor = torch.tensor(y).float().to(device)


In [ ]:
import torch.nn as nn

# Define your classifier (same as before)
class CamembertSentenceClassifier(nn.Module):
    def __init__(self, hidden_dim=768, mid_dim=256, dropout=0.1):
        super().__init__()
        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim, mid_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(mid_dim, 1),      # Binary classification
            nn.Sigmoid()                # Output in [0, 1]
        )

    def forward(self, x):
        return self.classifier(x)  # [batch_size, 1]

# Initialize the model
model = CamembertSentenceClassifier().to(device)

# Loss function for binary classification
loss_fn = nn.BCELoss()

# Optimizer (only for the classifier layers)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Training loop
model.train()
epochs = 10

for epoch in range(epochs):
    optimizer.zero_grad()

    # Forward pass
    outputs = model(X_tensor)  # Shape: [batch_size, 1]
    
    # Compute loss
    loss = loss_fn(outputs.squeeze(), y_tensor)

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Print loss
    if (epoch + 1) % 2 == 0:
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item():.4f}")
